In [1]:
import numpy as np
import pandas as pd 
import os
import spacy
import matplotlib.pyplot as plt
import docx

In [2]:
df = pd.read_json("json_training_agreementvalue_updated.json")
training_data = []
for i, one_ent in enumerate(df.annotations.values):
    lis_record =[]
    entities = {}
    for one_ent_ in one_ent[0]['result']:
        lis_single = []
        lis_single.append(one_ent_['value']['start'])
        lis_single.append(one_ent_['value']['end'])
        lis_single.append(one_ent_['value']['labels'][0])
        lis_record.append(tuple(lis_single))
        entities['entities']=lis_record
    training_data.append((list(df.data[i].values())[0], entities))

In [3]:
from spacy.training import Example
import random

TRAIN_DATA=training_data
#load model
nlp = spacy.blank('en')
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner', last=True)
#adding labels
for _, labels in TRAIN_DATA:
     for ent in labels.get('entities'):
        ner.add_label(ent[2])
#adding task to piplines
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
count=0
loss_val = []
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for i in range(100):
        print("iteration :" ,i," Start")
        random.shuffle(TRAIN_DATA)
        losses = {}
        #start training
        for text, annotations in TRAIN_DATA:
            doc = nlp.make_doc(text)
            try:
                example = Example.from_dict(doc, annotations)
                nlp.update(
                [example], 
                drop=0.3, 
                sgd=optimizer,
                losses=losses)
            except:
                continue

        loss_val.append(list(losses.values())[0])
        print("Loss in iteration :",i,"loss : ",list(losses.values())[0])

iteration : 0  Start


/Users/sanghavi/opt/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "Rental Agreement

This agreement made this 19" day..." with entities "[(43, 66, 'AGREEMENTDATE'), (75, 79, 'PARTYTWO'), ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/sanghavi/opt/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "RENTAL AGREEMENT

This rental agreement is made an..." with entities "[(76, 97, 'AGREEMENTDATE'), (115, 133, 'PARTYONE')...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/Users/sanghavi/opt/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.p

Loss in iteration : 0 loss :  11064.395686440748
iteration : 1  Start
Loss in iteration : 1 loss :  950.9806710943905
iteration : 2  Start
Loss in iteration : 2 loss :  506.32957723073014
iteration : 3  Start
Loss in iteration : 3 loss :  2077.065798469442
iteration : 4  Start
Loss in iteration : 4 loss :  2386.7730576803056
iteration : 5  Start
Loss in iteration : 5 loss :  1563.7239600993432
iteration : 6  Start
Loss in iteration : 6 loss :  1001.8584060508172
iteration : 7  Start
Loss in iteration : 7 loss :  2563.71523087649
iteration : 8  Start
Loss in iteration : 8 loss :  820.4350366739274
iteration : 9  Start
Loss in iteration : 9 loss :  840.3715475172422
iteration : 10  Start
Loss in iteration : 10 loss :  589.2791202420422
iteration : 11  Start
Loss in iteration : 11 loss :  609.8056191005119
iteration : 12  Start
Loss in iteration : 12 loss :  470.6697846159008
iteration : 13  Start
Loss in iteration : 13 loss :  1267.744116350325
iteration : 14  Start
Loss in iteration : 1

In [4]:
# training data i=100,drop =0.3
for text, _ in training_data:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('SUHAIMI BIN AG. AMAN', 'PARTYONE'), ('NOOR AZEENAH BINTI MICHAEL', 'PARTYTWO'), ('450', 'AGREEMENTVALUE'), ('1st July 2014', 'STARTDATE'), ('one (1) year', 'TERM'), ('3 0-day', 'NOTICE')]
Entities [('23 rd day of May Two Thousand Eleven (23- 05-2011)', 'AGREEMENTDATE'), ('M.V. Srinivas & M.V. Madhumathi', 'PARTYTWO'), ('M.V. Thirumalesh', 'PARTYTWO'), ('Rs. 4,200/- (Rupees Four Thousand\n\nTwo Hundred Only)', 'AGREEMENTVALUE'), ('Three months', 'NOTICE'), ('11 (Eleven) Months', 'TERM')]
Entities [('lst of May 2010', 'AGREEMENTDATE'), ('Mr. Balaji.R', 'PARTYONE'), ('Mr.Kartheek R', 'PARTYTWO'), ('Rs.3800/- (Rupees Thirty Eight Thousand Only)', 'AGREEMENTVALUE'), ('eleven (11) months', 'TERM'), ('one month', 'NOTICE')]
Entities [('LEE MING', 'PARTYONE'), ('JIMMY SINAU', 'PARTYTWO'), ('1-8-2011', 'STARTDATE'), ('1-8-2012', 'ENDDATE'), ('450', 'AGREEMENTVALUE')]
Entities [('1st day of September 2011 (1-3-2011)', 'AGREEMENTDATE'), ('k.Sarguneswaran', 'PARTYONE'), ('R.Mathankumar

Entities [('5th day of April, 2012', 'TERM'), ('Mrs, PADMAVATI', 'PARTYONE'), ('Mrs. SANGHAMITRA PARIDA', 'PARTYTWO'), ('Rs.8000', 'AGREEMENTVALUE'), ('(11) Months', 'TERM'), ('5th April of 2012', 'STARTDATE'), ('one-month', 'NOTICE')]
Entities [('14th day of July 2010', 'AGREEMENTDATE'), ('Mr. P. JohnsonRavikumar', 'PARTYONE'), ('Mr. Saravanan BV', 'PARTYTWO'), ('11 months', 'TERM'), ('01.04.2010', 'STARTDATE'), ('30.03.2011', 'ENDDATE'), ('Rs. 10,000/- (Rupees Ten Thousand only)', 'AGREEMENTVALUE'), ('three months', 'NOTICE')]
Entities [('19th day of August 2013', 'AGREEMENTDATE'), ('Mrs. M. Geetha,', 'PARTYONE'), ('M/s. Siruvani Traders Private Limited', 'PARTYTWO'), ('11 Months', 'TERM'), ('August 2014', 'STARTDATE'), ('Rs.2500/-', 'AGREEMENTVALUE'), ('4 months', 'NOTICE')]
Entities [('March 22nd day 2013 (TWO THOUSAND Thirteen)', 'AGREEMENTDATE'), ('Mr.Kamal', 'PARTYONE'), ('Mr.V.Arun Kumar', 'PARTYTWO'), ('12 months', 'TERM'), ('23.03.13', 'STARTDATE'), ('23.03.14', 'ENDDATE'), (

In [5]:
#training scores i=100, drop =0.3
from spacy.scorer import Scorer
examples = []

for input_, annotations in training_data:
        doc = nlp(input_)
        try:
            example = Example.from_dict(doc, annotations)
            examples.append(example)
        except:
            continue

scorer_ = Scorer(nlp)
scores = scorer_.score(examples)

print(scores)

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.9762845849802372, 'ents_r': 0.988, 'ents_f': 0.9821073558648111, 'ents_per_type': {'PARTYONE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'PARTYTWO': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'AGREEMENTVALUE': {'p': 0.9473684210526315, 'r': 0.9473684210526315, 'f': 0.9473684210526315}, 'STARTDATE': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'TERM': {'p': 0.90625, 'r': 1.0, 'f': 0.9508196721311475}, 'NOTICE': {'p': 0.96875, 'r': 1.0, 'f': 0.9841269841269841}, 'AGREEMENTDATE': {'p': 1.0, 'r': 0.9696969696969697, 'f': 0.9846153846153847}, 'ENDDATE': {'p': 1.0, 'r': 1.0, 'f': 1.0}}}


In [6]:
#preparing for validation data
df_v = pd.read_json("json_validation_agreementvalue.json")
validation_data = []
for i, one_ent in enumerate(df_v.annotations.values):
    lis_record =[]
    entities = {}
    for one_ent_ in one_ent[0]['result']:
        lis_single = []
        lis_single.append(one_ent_['value']['start'])
        lis_single.append(one_ent_['value']['end'])
        lis_single.append(one_ent_['value']['labels'][0])
        lis_record.append(tuple(lis_single))
        entities['entities']=lis_record
    validation_data.append((list(df_v.data[i].values())[0], entities))

In [7]:
validation_data

[('Registration and Stamps Department\n\neodcb&ja^ asraoS Date of execution\n\nZjUj, stensjoi abcjAd©.\n\nTotal stamp duty paid Rs.\n\n\tCj3J2|3Ct5D\towSFliS\n\n\tedeS ?Joa^\t152 sfofisedx 2003\n\najssoS 09-05—2003d d,5fad sSao/fifiKBOPSTAMP DUTY KARNATAKA\n\nc$j5Coz3f$ 3oc>rtjs s&o^c^ost ^C7e)©c> &sd: &&. 2/\n\n\tds\tafss&de dTOjSczSrt eroaiaieef«^*^x\n\nThis sheet can be used for any document _ . , ,\n\nrossssSrfrS d,\n\nWR E N T A L AGREEMENT”\n\nThis Rental Agreement is made and Executed on\n\nat Bangalore, by and Between:- wef 1/4/2014\n\nMr .Giddappa S/o lute Nagappa\n\nD No.134 2nd cross\n\nShunthi L/o Rumamurthynagar\n\nBangalore 560016\n\nHereinafter Called the "OWNER" of the One Part: Ms Pottumurthi Udayalaxmi W/o Ramakr ishna Kajjam\n\nH No.27-7 Main Bazar,\n\nMiryalaguda, i, Nalgonda Dist A p 503207.\n\nHereinafter called the "TENANT" of the Other Part witnesseth as follows terms and conditions.\n\n\t1. Whereas the Tenant has paid Advance amount of Rs.40,000/- ( Forty thous

In [8]:
#validation scores for i=100, drop =0.3
from spacy.scorer import Scorer
examples = []
for input_, annotations in validation_data:
        doc = nlp(input_)
        try:
            example = Example.from_dict(doc, annotations)
            examples.append(example)
        except:
            continue
            
scorer_ = Scorer(nlp)
scores = scorer_.score(examples)

print(scores)

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.8571428571428571, 'ents_r': 0.7777777777777778, 'ents_f': 0.8155339805825242, 'ents_per_type': {'AGREEMENTDATE': {'p': 1.0, 'r': 0.8571428571428571, 'f': 0.923076923076923}, 'PARTYONE': {'p': 0.7777777777777778, 'r': 0.875, 'f': 0.823529411764706}, 'PARTYTWO': {'p': 0.8571428571428571, 'r': 0.75, 'f': 0.7999999999999999}, 'AGREEMENTVALUE': {'p': 0.7142857142857143, 'r': 0.625, 'f': 0.6666666666666666}, 'TERM': {'p': 1.0, 'r': 0.875, 'f': 0.9333333333333333}, 'STARTDATE': {'p': 0.7142857142857143, 'r': 0.7142857142857143, 'f': 0.7142857142857143}, 'NOTICE': {'p': 1.0, 'r': 0.8571428571428571, 'f': 0.923076923076923}, 'ENDDATE': {'p': 0.0, 'r': 0.0, 'f': 0.0}}}


/Users/sanghavi/opt/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "RENTAL AGREEMENT

This Rental Agreement is made an..." with entities "[(72, 96, 'AGREEMENTDATE'), (133, 151, 'PARTYONE')...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


In [9]:
for text, _ in validation_data:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('1/4/2014', 'AGREEMENTDATE'), ('Mr .Giddappa', 'PARTYONE'), ('Ms Pottumurthi Udayalaxmi', 'PARTYTWO'), ('thousand three hundred only\t', 'AGREEMENTVALUE'), ('11(Eleven) months', 'TERM'), ('date of Agreement', 'STARTDATE'), ('Two Month', 'NOTICE')]
Entities [('15th of December 2012', 'AGREEMENTDATE'), ('SRI VYSHNAVI DAIRY SPECIALITIES Private Ltd.', 'PARTYTWO'), ('ll(eleven) months', 'TERM'), ('date of this agreement', 'STARTDATE'), ('one month', 'NOTICE')]
Entities [('1st May 2005', 'AGREEMENTDATE'), ('Mrs. S.Sakunthala', 'PARTYONE'), ('V.V.Ravi Kian', 'PARTYTWO'), ('11 (Eleven) months', 'TERM'), ('1st April 2010', 'STARTDATE'), ('Rs. 9,000/- (Nine thousand and two hundred rupees only)', 'AGREEMENTVALUE'), ('one month', 'NOTICE')]
Entities [('17th day of July two thousand\n\nforteen (17-07-2014)', 'AGREEMENTDATE'), ('Mr. SIMON PA', 'PARTYONE'), ('Mr. VAIRAVAN A', 'PARTYONE'), ('Rs. 9000/- (Rupees Nine Thousand Only)', 'AGREEMENTVALUE'), ('11 (Eleven) months', 'TERM'), ('07-0